# Linear Regression
--- 

Use this notebook for 5-tool analysis

5 tools vs. WAR  
    - vs. FG WAR  
    - vs. BBRef WAR  

<a href="https://en.wikipedia.org/wiki/Glossary_of_baseball_(0%E2%80%939)#5-tool_player" target="_blank">5 tool baseball player: wikipedia.com</a>

<a href="http://www.performancebaseball.net/html/5tools.html" target="_blank">5 tool baseball player: performancebaseball.net</a>

<a href="http://fivethirtyeight.com/features/the-toolsiest-player-of-them-all/" target="_blank">'Toolsiest player of them all': 538.com</a>

<a href="http://www.baseball-reference.com/about/war_explained.shtml" target="_blank">Baseball Reference WAR</a>

<a href="http://www.fangraphs.com/library/misc/war/" target="_blank">Fangraphs WAR</a>

<a href="https://en.wikipedia.org/wiki/Wins_Above_Replacement" target="_blank">Wikipedia WAR</a>

<a href="http://www.espn.com/blog/sweetspot/post/_/id/27050/what-we-talk-about-when-we-talk-about-war" target="_blank">ESPN WAR</a>

__WAR__ stands for _wins above replacement_, and is a very popular sabermetric . According to <a href="http://www.espn.com/blog/sweetspot/post/_/id/27050/what-we-talk-about-when-we-talk-about-war" target="_blank">ESPN:</a>

> It [WAR] attempts to evaluate a player's total contribution -- batting, fielding, baserunning, pitching -- to his team. It is expressed in terms of wins over the perceived value of a replacement-level player at each position, essentially a Triple-A player. (A team of replacement-level players would be expected to win about 50 games.)

Players with high WAR values can be said to contribute more value to their teams than players with low WAR values

Since players with high WAR values are said to contribute more value (in the form of wins) to their teams than players with low WAR values, we might expect to see teams with rosters composed of high-WAR players to win more games in a season than teams with rosters composed of low-WAR players. In this analysis, I will use two common  

<a href="http://www.baseball-reference.com/about/war_explained.shtml" target="_blank">Baseball Reference</a> and <a href="http://www.fangraphs.com/library/misc/war/" target="_blank">Fangraphs </a> are two popular websites that have developed their own formulas for calculating WAR

In [10]:
ls data/baseballdatabank-master/core

AllstarFull.csv          Fielding.csv             Pitching.csv
Appearances.csv          FieldingOF.csv           PitchingPost.csv
AwardsManagers.csv       FieldingPost.csv         Salaries.csv
AwardsPlayers.csv        HallOfFame.csv           Schools.csv
AwardsShareManagers.csv  HomeGames.csv            SeriesPost.csv
AwardsSharePlayers.csv   Managers.csv             Teams.csv
Batting.csv              ManagersHalf.csv         TeamsFranchises.csv
BattingPost.csv          Master.csv               TeamsHalf.csv
CollegePlaying.csv       Parks.csv                readme2014.txt


In [4]:
import numpy as np
import pandas as pd

In [42]:
# import lahman data
lf = pd.read_csv('data/baseballdatabank-master/core/Fielding.csv')
lb = pd.read_csv('data/baseballdatabank-master/core/Batting.csv')
lp = pd.read_csv('data/baseballdatabank-master/core/Pitching.csv')
lt = pd.read_csv('data/baseballdatabank-master/core/Teams.csv')
                 
# import fangraphs data
ff = pd.read_csv('data/FG_fielding.csv',encoding='utf-16')
fb = pd.read_csv('data/FG_offense.csv')
fp = pd.read_csv('data/FG_pitching.csv')

CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

predictors:  
- bat for average = batting avg  
- bat for power = ISO  
- arm = FG_def  
- glove = FG_def  
- speed = FG_UBR | FG_Spd  

In [40]:
ff.rename(columns={'\"Season\"': 'Year','Team':'Team2'}, inplace=True)
ff.columns
# data = ff.loc[:,['Season','Team','Def']]
# data.head()

Index([u'﻿"Season"', u'Team2', u'Inn', u'rSB', u'rGDP', u'rARM', u'rGFP',
       u'rPM', u'DRS', u'BIZ', u'Plays', u'RZR', u'OOZ', u'CPP', u'RPP',
       u'TZL', u'FSR', u'ARM', u'DPR', u'RngR', u'ErrR', u'UZR', u'UZR/150',
       u'Def'],
      dtype='object')

In [19]:
ff.columns

Index([u'﻿"Season"', u'Team', u'Inn', u'rSB', u'rGDP', u'rARM', u'rGFP',
       u'rPM', u'DRS', u'BIZ', u'Plays', u'RZR', u'OOZ', u'CPP', u'RPP',
       u'TZL', u'FSR', u'ARM', u'DPR', u'RngR', u'ErrR', u'UZR', u'UZR/150',
       u'Def'],
      dtype='object')

In [ ]:
ff.rename